In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 05: Task Pattern (A2A)

This notebook demonstrates the **Task Communication Pattern** using the **Agent-to-Agent (A2A) protocol** - an open standard that enables one agent to delegate work to another agent by creating and managing tasks.

## Understanding the Task Pattern

The **Task Pattern** is a communication approach where:
- One agent (the **delegator**) creates a task for another agent (the **worker**)
- The worker agent processes the task independently
- The worker updates the task with results/artifacts when complete
- Communication happens asynchronously via task status and artifacts

## A2A Protocol: An Implementation of the Task Pattern

The **A2A (Agent-to-Agent) protocol**, introduced by Google, is a standardized HTTP-based implementation of the Task Pattern. It defines:
- How to create tasks with specific goals
- How agents communicate task status
- How to exchange artifacts (results/data)
- A standard message format for interoperability

## Task Pattern Architecture

```mermaid
sequenceDiagram
    participant User as 👤 User
    participant Agent1 as 🤖 Agent 1<br/>Financial Assistant
    participant A2A as 📡 A2A Server<br/>(localhost:8001)
    participant Agent2 as 📊 Agent 2<br/>Stock Analysis

    User->>Agent1: Request stock analysis
    
    Agent1->>A2A: POST / (message/send)<br/>Create Task
    A2A-->>Agent1: Task ID
    
    Note over A2A,Agent2: Agent 2 processes task<br/>(web search, analysis)
    
    Agent1->>A2A: POST / (tasks/get)
    A2A-->>Agent1: Status: completed + StockReport
    
    Agent1-->>User: ✅ Present analysis
```

> **Note:** This diagram shows the **polling** approach. A2A also supports **streaming (SSE)** for real-time updates and **push notifications (webhooks)** for server-initiated callbacks.

**Key Components:**
1. **Task Creation**: Agent 1 creates a task with a specific goal
2. **Task Delegation**: A2A protocol delivers the task to Agent 2
3. **Task Processing**: Agent 2 works on the task independently
4. **Task Updates**: Agent 2 updates task status during processing
5. **Artifact Exchange**: Agent 2 attaches results as artifacts to the task
6. **Task Completion**: Agent 1 retrieves the completed task with artifacts

**⚠️ IMPORTANT**: Before running this notebook, start the A2A stock analysis server in a separate terminal:

```bash
uvicorn lecture05.stock_a2a_server:app --host 0.0.0.0 --port 8001
```

**Reference**: [Pydantic AI A2A Documentation](https://ai.pydantic.dev/a2a/)


## A2A Server Code (Agent 2)

The server code runs separately in a terminal. It's remarkably simple - just one line to expose an agent as an A2A server:

```python
# File: lecture05/stock_a2a_server.py
# Run with: uvicorn lecture05.stock_a2a_server:app --port 8001

from pydantic_ai import Agent
from common.models import StockReport
from common.tools import web_search_tool
from common.utils import create_agent_model

stock_analysis_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    output_type=StockReport,
    system_prompt="You are an expert financial analyst..."
)

# This single line exposes the agent as an A2A server!
app = stock_analysis_agent.to_a2a()
```

In [ ]:
# === A2A CLIENT CODE (Agent 1 → Agent 2) ===
# This shows how Agent 1 communicates with Agent 2 via A2A protocol

import httpx
import uuid
import asyncio

A2A_SERVER = "http://localhost:8001"


async def request_stock_analysis_a2a(symbol: str) -> dict:
    """
    Request stock analysis via A2A protocol.
    A2A uses JSON-RPC 2.0 over HTTP.
    """
    # Step 1: Send message to create a task
    a2a_request = {
        "jsonrpc": "2.0",
        "method": "message/send",
        "params": {
            "message": {
                "role": "user",
                "parts": [{"kind": "text", "text": f"Analyze stock {symbol}"}],
                "kind": "message",
                "messageId": str(uuid.uuid4()),
            },
        },
        "id": str(uuid.uuid4()),
    }

    async with httpx.AsyncClient(timeout=120.0) as client:
        # Create the task
        response = await client.post(f"{A2A_SERVER}/", json=a2a_request)
        task_id = response.json()["result"]["id"]
        print(f"📤 Task created: {task_id}")

        # Step 2: Poll for completion (A2A also supports SSE streaming)
        while True:
            await asyncio.sleep(1)
            status_request = {
                "jsonrpc": "2.0",
                "method": "tasks/get",
                "params": {"id": task_id},
                "id": str(uuid.uuid4()),
            }
            status = await client.post(f"{A2A_SERVER}/", json=status_request)
            task = status.json()["result"]
            state = task["status"]["state"]
            print(f"🔄 Task status: {state}")

            if state == "completed":
                print("✅ Task completed!")
                return task


print("A2A client code loaded. See request_stock_analysis_a2a() function above.")

A2A client code loaded. See request_stock_analysis_a2a() function above.


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging
from lecture05.agent import a2a_stock_analysis_tool  # Pre-built A2A tool

# Initialize logging
setup_logging()

In [4]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="task-pattern-a2a-lecture05",
    start_message="🚀 Lecture 05 - Task Pattern (A2A) Notebook Started",
)

In [ ]:
# Create financial assistant with Task Pattern (A2A) delegation capability
# The a2a_stock_analysis_tool implements the Task Pattern:
# 1. Creates a task for the stock analysis agent
# 2. The worker agent processes the task independently
# 3. Results are returned as task artifacts via A2A protocol
a2a_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, a2a_stock_analysis_tool],
    system_prompt="""
You are a knowledgeable financial assistant using the Task Pattern for delegation.

When answering questions:
1. Use web_search for general financial information
2. Use request_stock_analysis_a2a to delegate stock analysis tasks
   - This creates a TASK for the specialist stock analysis agent
   - The task is sent via A2A protocol (HTTP) to a worker agent
   - The worker processes the task and returns results as artifacts
   - The worker agent runs as a separate A2A server at localhost:8001
3. Explain to users when you're delegating tasks to specialist agents

This demonstrates the Task Pattern where you delegate specialized work to other agents.

Keep your responses informative and concise.
""",
)

In [ ]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

Question: What is the current price of Bitcoin?

Response:
The current price of Bitcoin is approximately $92,900 USD. Prices may vary slightly across different exchanges, but this reflects the latest real-time value. For the most up-to-date price, you can also visit sites like TradingView or Coinbase.


In [ ]:
# Ask for detailed stock analysis (uses A2A protocol)
# This will:
# 1. Send HTTP request to A2A server at localhost:8001
# 2. Use A2A protocol message format
# 3. Receive StockReport from A2A server
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await a2a_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

2026-01-05 20:18:44 - lecture05.agent - INFO - A2A Protocol: Preparing request for stock analysis of AAPL
2026-01-05 20:18:44 - lecture05.agent - INFO - A2A Protocol: Sending HTTP request to http://localhost:8001/
2026-01-05 20:18:44 - lecture05.agent - INFO - A2A Protocol: Task created with ID ec42862e-0549-4210-a748-59a0c5535710, polling for completion...
2026-01-05 20:18:56 - lecture05.agent - INFO - A2A Protocol: Task ec42862e-0549-4210-a748-59a0c5535710 completed successfully
2026-01-05 20:18:56 - lecture05.agent - INFO - A2A Protocol: Successfully extracted StockReport from artifacts for AAPL


Question: Can you give me a detailed analysis of Apple stock (AAPL)?

Response:
Here’s a detailed analysis of Apple Inc. (AAPL):

Executive Summary:
Apple continues to lead the technology sector, reporting record revenues and robust growth momentum, especially in iPhone sales and expanding services (like subscriptions and AI-driven features). The company is investing heavily in artificial intelligence and new product categories, which supports its long-term growth prospects.

Recent News Highlights:
- Apple projects 10%-12% revenue growth in Q1 2026, led by iPhone sales.
- Record FY2025 revenue: $416 billion, the highest in Apple’s history.
- Analyst consensus for 2026: Moderate buy, with target prices between $299 and $350.
- Raymond James downgraded Apple to "Market Perform" due to high valuation and limited short-term upside.
- Outperformed most AI stocks recently, but lags the S&P 500’s one-year return.

Financial Highlights:
- FY2025 revenue: $416 billion (Q4 YoY growth: +8%)
- iP